# Instructions
**VERY IMPORTANT: This must be run AFTER you've updated the stockrooms in ServiceNow, but  BEFORE the items are delivered/assigned.**

One time setup: At the top of the page, click "Copy to Drive". Once you have a copy, save that link to your bookmarks. You'll have to run the script from your copy otherwise you can't use the "Run all" button.

I recommend you run this before your delivery, save the output in notepad, and update the work notes when you get back.

How to run:
* ServiceNow Stuff
  1. Search all your barcodes and S/N's in ServiceNow.
  2. Select all rows, click the three dots next to any column header, select export, and choose CSV.
  3. It should save under the name "alm_hardware.csv". Take note of where it saves (probably downloads).
* Colab/Script Stuff
  4. Click 'Run all' at the top of this page.
  5. Scroll down to the 3rd cell and click 'Choose Files' button.
  6. Select your "alm_hardware.csv".
  7. Copy output at the bottom of the page and paste into your work notes in ServiceNow.




In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
# Class to store an item being delivered. Help with sorting.
class Delivery:
  def __init__(self, barcode, display_name, stockroom):
    self.barcode = barcode
    self.display_name = display_name
    self.stockroom = stockroom

  def __lt__(self, other):
    if (self.display_name != other.display_name):
      return self.display_name < other.display_name
    return self.barcode < other.barcode

  def __str__(self):
    return f"{self.barcode}: {self.display_name} at {self.stockroom}"

  def __repr__(self):
    return f"{self.barcode}: {self.display_name} at {self.stockroom}"

In [ ]:
# Delete any existing uploaded files
!rm -f /content/*.csv

# Upload files
uploaded = files.upload()
df = pd.read_csv("alm_hardware.csv")

# Check to make sure no stockroom is NaN
required_cols = ["stockroom"]
if df[required_cols].isna().any().any():
    raise ValueError("❌ One or more items are not assigned stockrooms. Either you forgot to update the item's stockroom or it's been assigned already. Please fix, re-generate the CSV, and re-upload.")
df = df.dropna(subset="stockroom")

Saving alm_hardware.csv to alm_hardware.csv


In [ ]:
delivery_list = []
for index, row in df.iterrows():
  delivery_list.append(Delivery(barcode=row['asset_tag'], display_name=row['model.display_name'], stockroom=row['stockroom']))

In [ ]:
# Get unique location names
locations = df['stockroom'].unique()

# Build a list of deliveries for each location and store in dictionary
location_lists = {}
for location in locations:
  location_lists[location] = sorted([delivery for delivery in delivery_list if delivery.stockroom == location])

# Get unique display names
display_names = sorted(df['model.display_name'].unique())

for location in locations:
  print(f"---------- {location} ----------")
  for name in display_names:
    # Generate list of devices up front to get count
    devices = [delivery for delivery in location_lists[location] if delivery.display_name == name]
    # Print devices with non-zero qty
    if (len(devices)):
      print(f"[({len(devices)}x) {name}]: ")
      for device in devices:
        print(device.barcode)
  print()


---------- ACC  A112 ----------
[(1x) HP 24 G5 LED 6N6E9AA]: 
221260
[(7x) HP 24 LED 524pf 9D9L6UT]: 
235261
241410
241420
241421
241422
241423
241451
[(6x) HP EliteBook 8 G1i 16-C75WJUC]: 
241534
241535
241536
241537
241538
241539
[(8x) HP EliteDesk 8 G1i Mini-AR6D2AV]: 
242472
242473
242474
242475
242476
242477
242478
242479
[(6x) HP USB-C Dock G5 5TW10AA]: 
241090
241094
241104
241121
241123
241134

